<h1 style='color: #c52b10; font-size: 34px; font-weight: bold;'>Sistema de Recomendação de Filmes

</h1>


# <font color="#10c586" style="font-size: 30px;">Objetivo</font>
<hr style="border: 2px solid #10c586;">

 <p style='font-size: 23px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i><b> Desenvolver um sistema de recomendação de filmes com base em características como gênero, palavras-chave, popularidade e classificação média, etc. </b></i>     
</p>

---

# <font color="#10c586" style="font-size: 30px;">Roteiro</font>
<hr style="border: 2px solid #10c586;">

<h1 style='color: black; font-size:22px; font-weight: bold;'> 1. Pré-processamento de dados
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'>2. Análise exploratória de dados

</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
3. Modelagem de dados
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
4. Comunicação de Resultados
</h1>

---


# <font color="#10c586" style="font-size: 30px;">1. Pré-processamento de dados</font>
<hr style="border: 2px solid #10c586;">

In [132]:
# IMPORTANDO BIBLIOTECAS

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [133]:
# IMPORTANDO BASE DE DADOS

df = pd.read_csv('tmdb_5000_movies.csv')

df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [134]:
# RENOMEANDO AS COLUNAS

traducao = {
    'budget': 'Orcamento',
    'genres': 'Gêneros',
    'homepage': 'Página inicial',
    'id': 'ID',
    'keywords': 'Palavras-chave',
    'original_language': 'Idioma original',
    'original_title': 'Título original',
    'overview': 'Visão geral',
    'popularity': 'Popularidade',
    'production_companies': 'Empresas de produção',
    'production_countries': 'Países de produção',
    'release_date': 'Data de lançamento',
    'revenue': 'Receita',
    'runtime': 'Duração',
    'spoken_languages': 'Idiomas falados',
    'status': 'Status',
    'tagline': 'Slogan',
    'title': 'Título',
    'vote_average': 'Media_de_votos',
    'vote_count': 'Contagem_de_votos'
}

df = df.rename(columns=traducao)
df.head(2)

,Orcamento,Gêneros,Página inicial,ID,Palavras-chave,Idioma original,Título original,Visão geral,Popularidade,Empresas de produção,Países de produção,Data de lançamento,Receita,Duração,Idiomas falados,Status,Slogan,Título,Media_de_votos,Contagem_de_votos
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


---
## <font color=#c5106e style="font-size: 30px;">Transformar coluna de Data de lançamento em datetime</font>

---

In [135]:
# DANDO UMA OLHADA NA COLUNA DE LANÇAMENTOS

display(type(df['Data de lançamento'][0]))

#Converter representação de data em formato de string para um objeto de data no Python
 
from datetime import datetime

df['Data de lançamento'] = pd.to_datetime(df['Data de lançamento']) 

display(type(df['Data de lançamento'][0]))

str

pandas._libs.tslibs.timestamps.Timestamp

---
## <font color=#c5106e style="font-size: 30px;">Conformar a coluna de Gêneros para a forma que queremos</font>

---

In [136]:
nomes = df['Título original']
nomes = list(nomes)

generos = df['Gêneros']
generos = list(generos)


import json

# Lista de filmes
lista_filmes = nomes

# Lista de gêneros cinematográficos
lista_generos = generos

# Função para extrair os nomes dos gêneros de uma string em formato JSON
def extrair_generos(generos_str):
    generos_json = json.loads(generos_str)
    return [genero['name'] for genero in generos_json]

# Lista de gêneros cinematográficos para cada filme

lista_generos_filmes = [extrair_generos(generos_str) for generos_str in lista_generos]
df['Gêneros'] = lista_generos_filmes

```python
# Transforma a lista de listas em uma lista simples
lista_simples = [item for sublist in lista_generos_filmes for item in sublist]

# Obtém os valores únicos da lista
valores_unicos = set(lista_simples)

df['Gêneros'] = lista_simples[:4803]
```

---
## <font color=#c5106e style="font-size: 30px;">Removendo Dados inconsistentes</font>

---

In [137]:
df.describe()

,Orcamento,ID,Popularidade,Receita,Duração,Media_de_votos,Contagem_de_votos
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


---
## <font color=#c5106e style="font-size: 30px;">Não vamos querer nenhum filme com:</font>
---

*  <font color=#c5106e style="font-size: 28px;">Orçamento Zero</font>

---

* <font color=#c5106e style="font-size: 28px;">Receita Zero</font>
---

* <font color=#c5106e style="font-size: 28px;">Duração Zero</font>
---

* <font color=#c5106e style="font-size: 28px;">Média de votos Zero</font>
---

* <font color=#c5106e style="font-size: 28px;">Contagem de votos Zero</font>

---

In [138]:
df = df.query('Orcamento > 0 and Receita > 0 and Duração > 0 and Media_de_votos > 0 and Contagem_de_votos > 0')

---
## <font color=#c5106e style="font-size: 30px;">Mais de 75% dos filmes tem mais de 100 votos. Vamos filtrar</font>
---

In [140]:
df = df.query('Contagem_de_votos >= 100') # para assim só termos filmes com estatísticas relevantes de nota 